<a href="https://colab.research.google.com/github/EdithOroche/IA/blob/main/Estimando_Distribuciones_Cont%C3%ADnuas_de_Datos_SS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install distfit

## 📊 Distribuciones de probabilidad

Estas distribuciones se utilizan ampliamente en estadística, simulación y ciencia de datos para modelar fenómenos aleatorios .

---

### ✅ Distribuciones continuas

#### **Erlang**
- **Tipo:** Continua (caso especial de Gamma)  
- **Parámetros:** $k$ (entero > 0), $\lambda$ (tasa)  
- **Soporte:** $x \ge 0$  
- **Descripción:** Modela el tiempo hasta completar $k$ eventos exponenciales independientes.  
- **Aplicaciones:** teoría de colas, tiempos de servicio con varias etapas.  
$$
f(x)=\frac{\lambda^k x^{k-1}e^{-\lambda x}}{(k-1)!}
$$

---

#### **Exponencial**
- **Tipo:** Continua  
- **Parámetros:** $\lambda$ (tasa)  
- **Soporte:** $x \ge 0$  
- **Descripción:** Tiempo entre eventos de un proceso Poisson; **no memoria**.  
- **Aplicaciones:** tiempos entre llegadas, fallas de sistemas.  
$$
f(x)=\lambda e^{-\lambda x}
$$

---

#### **Normal**
- **Tipo:** Continua  
- **Parámetros:** $\mu$ (media), $\sigma^2$ (varianza)  
- **Soporte:** $-\infty < x < \infty$  
- **Descripción:** Simétrica en forma de campana; fenómenos naturales y errores de medición.  
- **Aplicaciones:** alturas, puntajes, errores experimentales.  
$$
f(x)=\frac{1}{\sqrt{2\pi\sigma^2}}\,\exp\!\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)
$$

---

#### **Uniforme continua**
- **Tipo:** Continua  
- **Parámetros:** $a$ (mín), $b$ (máx)  
- **Soporte:** $a \le x \le b$  
- **Descripción:** Todos los valores del intervalo son equiprobables.  
- **Aplicaciones:** simulación base, incertidumbre sin sesgo.  
$$
f(x)=\frac{1}{b-a},\quad a\le x\le b
$$

---

### 📌 Resumen rápido

| Distribución | Tipo | Modelo típico / Uso |
|--------------|------|---------------------|
| Erlang | Continua | Tiempos con múltiples etapas (colas) |
| Exponencial | Continua | Tiempo entre eventos (no memoria) |
| Normal | Continua | Fenómenos naturales y errores |
| Uniforme | Continua | Incertidumbre sin sesgo |
```

#Generaremos muestras iniciales cada una de tamaño 100, que simularán las tomas reales, para luego descubrir que distribución se ajusta a ellos y realizar una comparación.

In [ ]:
import numpy as np

rng = np.random.default_rng(42)   # reproducibilidad
N = 100

# --- Continuas ---
# Erlang(k, λ) ≡ Gamma(shape=k, scale=1/λ)
k_erlang, lam_erlang = 3, 0.5
erlang = rng.gamma(shape=k_erlang, scale=1/lam_erlang, size=N)

# Exponencial(λ)
lam_exp = 0.4
exponencial = rng.exponential(scale=1/lam_exp, size=N)

# Normal(μ, σ)
mu, sigma = 10.0, 2.0
normal = rng.normal(loc=mu, scale=sigma, size=N)

# Uniforme continua [a, b]
a_u, b_u = 0.0, 1.0
uniforme = rng.uniform(low=a_u, high=b_u, size=N)

# (Opcional) muestra rápida de las primeras entradas
for name, arr in [
    ("erlang", erlang), ("exponencial", exponencial), ("normal", normal),
    ("uniforme", uniforme)
]:
    print(f"{name}: {arr[:10]}")


#Gráficas de la Distribuciones de Datos

In [ ]:
# === DISTPLOTS PARA VARIABLES CONTINUAS
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# Diccionario de datos (nombre -> array)
data_dict = {
    "erlang": erlang,
    "exponencial": exponencial,
    "normal": normal,
    "uniforme": uniforme
}

fig, ax = plt.subplots(2, 2, figsize=(8,6))
for a, (name, x) in zip(ax.ravel(), data_dict.items()):
    xs = np.linspace(np.min(x), np.max(x), 200); a.hist(x, bins='fd', density=True, alpha=.6); a.plot(xs, gaussian_kde(x)(xs)); a.set_title(name)
plt.tight_layout()

# El siguiente bloque de código utiliza la librería **distfit** para identificar qué distribución teórica se ajusta mejor a un conjunto de datos (`erlang`) y obtener sus parámetros estimados:

In [ ]:
import numpy as np
from distfit import distfit
d = distfit(todf=False)
data = np.array(erlang)
d.fit_transform(data, verbose=False)
print(d.model)
d.plot()
print(d.model['params'])

## 📘 Interpretación del resultado de `distfit`

El ajuste indica que **la mejor distribución** para los datos es **Beta** (con desplazamiento y escala).

### 🔑 Campos clave del diccionario

- **name: 'beta'**  
  Distribución Beta (continua), aquí usada en su versión desplazada/escala.

- **arg: (α, β)**  
  Parámetros de **forma** de la Beta estándar:  
  - α = 2.2593065894092117  
  - β = 11.724016207811122

- **loc, scale**  
  Desplazamiento y escala que transforman la Beta de \([0,1]\) a \([loc, loc+scale]\):  
  - loc = 0.6839244496148762  
  - scale = 31.373186019411  
  - Soporte: $[0.6839,\ 32.0571]$ aprox.

- **params: (α, β, loc, scale)**  
  Paquete completo listo para usar con `scipy.stats.beta`.

- **model: <...rv_continuous_frozen>**  
  Objeto “congelado” de SciPy: permite llamar directamente `rvs/pdf/cdf/ppf` con esos parámetros.

- **score (RSS)**  
  `score = 0.003079...` es el **Residual Sum of Squares** del ajuste (menor = mejor).


# Generaremos 1000 datos sintéticos a partir de la distribución a la que se ajustan los datos.
## Nota: A partir de 100 datos, estimamos una distribución para poder generar más datos (1000).

In [ ]:
from scipy import stats

name = d.model.get('name', d.model.get('distribution'))
params = d.model.get('params', tuple(d.model.get('arg', ())) + (d.model.get('loc',0), d.model.get('scale',1)))
synthetic = getattr(stats, name)(*params).rvs(size=1000, random_state=42)

# Ahora, graicaremos los datos reales (los 100 generados originalmente) y los sintéticos (a partir de la distribución estimada) y compararemos.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(data, bins=30, density=True, alpha=0.6, label="Real")
plt.hist(synthetic, bins=30, density=True, alpha=0.6, label="Simulada")
plt.legend()
plt.show()

# Tarea:
Estimar las distribuciones y parámetros para las muestras

1.   Estimar las distribuciones y parámetros para las muestras almacenadas en las variables exponencial, normal y uniforme.
2.   Comparar si la distribución que originó la muestra es igual a la distribución estimada.
3.   Comparar cualitativamente si las distribuciones graficadas (real vs sintética) son similares.
4.   Responder: ¿Qué es el SCORE o RSS que calcula el distfit? ¿Con respecto a qué se calcula?
5.   ¿Qué distribución sigue el siguiente conjunto de datos?


> 191.088 178.781 199.534 191.382 173.618 193.244 185.665 192.927 175.524 189.795 176.583 187.396 198.267 181.183 186.406 197.473 193.006 192.881 187.241 187.850 193.267 192.581 195.991 200.364 188.597 188.870 191.077 206.708 190.292 198.489 198.414 174.540 194.575 184.283 194.842 186.476 196.176 183.730 197.700 184.097 203.231 192.099 177.140 172.582 188.939 183.386 180.174 195.355 193.626 206.255



